<a href="https://colab.research.google.com/github/jmbaek/NLP_applications/blob/main/PretrainingBERT_PythonCodeTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 9.1 MB/s 
     |████████████████████████████████| 462 kB 50.5 MB/s 


In [2]:
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [26]:
bert_preprocess = hub.KerasLayer(preprocess_url)

In [27]:
text_test = ['that was a nice movie indeed', 'I love python programming']
text_processed = bert_preprocess(text_test)
text_processed.keys()

dict_keys(['input_word_ids', 'input_type_ids', 'input_mask'])

In [6]:
text_processed['input_word_ids']

<tf.Tensor: shape=(2, 128), dtype=int32, numpy=
array([[  101,  2008,  2001,  1037,  3835,  3185,  5262,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [28]:
bert_encoder = hub.KerasLayer(encoder_url)
bert_results = bert_encoder(text_processed)
bert_results.keys()

dict_keys(['default', 'encoder_outputs', 'pooled_output', 'sequence_output'])

In [29]:
bert_results['sequence_output']

<tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
array([[[ 0.08247545, -0.02013815,  0.14680883, ..., -0.05504194,
          0.21207508,  0.11869051],
        [-0.46663198, -0.24285477, -0.3423316 , ..., -0.2831246 ,
          1.3741698 ,  0.19046658],
        [-0.27740252, -0.57036436,  0.24329393, ...,  0.23598474,
          0.2682978 ,  0.26211837],
        ...,
        [ 0.17072722,  0.01922826,  0.5363991 , ...,  0.24514116,
          0.2635091 , -0.04776914],
        [ 0.21648622,  0.04456874,  0.547078  , ...,  0.2789396 ,
          0.2860774 , -0.07193862],
        [ 0.08261422,  0.07403944,  0.5833179 , ...,  0.26967448,
          0.19498406, -0.02856611]],

       [[-0.07900561,  0.36335143, -0.21101603, ..., -0.17183733,
          0.16299734,  0.6724263 ],
        [ 0.27883607,  0.43716288, -0.35764828, ..., -0.0446361 ,
          0.38315198,  0.58879817],
        [ 1.2037672 ,  1.0727026 ,  0.4840874 , ...,  0.24921   ,
          0.40730882,  0.4048173 ],
        ...,

In [30]:
bert_results['encoder_outputs'][-1] == bert_results['sequence_output']

<tf.Tensor: shape=(2, 128, 768), dtype=bool, numpy=
array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]]])>

## Spam detection

In [10]:
!mkdir -p  ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#Copy the API command from kaggle
!kaggle datasets download -d venky73/spam-mails-dataset
!unzip -q /content/spam-mails-dataset.zip -d data

  0% 0.00/1.86M [00:00<?, ?B/s]
100% 1.86M/1.86M [00:00<00:00, 145MB/s]


In [12]:
import tensorflow as tf
import tensorflow_hub as hup
import tensorflow_text as text
import pandas as pd

In [14]:
df = pd.read_csv("data/spam_ham_dataset.csv")
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [16]:
df.groupby('label').describe()

Unnamed: 0                                                         \
           count    mean          std     min      25%     50%      75%   
label                                                                     
ham       3672.0  1835.5  1060.159422     0.0   917.75  1835.5  2753.25   
spam      1499.0  4421.0   432.868340  3672.0  4046.50  4421.0  4795.50   

              label_num                                     
          max     count mean  std  min  25%  50%  75%  max  
label                                                       
ham    3671.0    3672.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
spam   5170.0    1499.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

In [17]:
df['label'].value_counts()

ham     3672
spam    1499
Name: label, dtype: int64

In [18]:
# Imbalance data treatment
df_spam = df[df['label'] == 'spam']
df_spam.shape


(1499, 4)

In [19]:
df_ham = df[df['label'] == 'ham']
df_ham.shape

(3672, 4)

In [20]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(1499, 4)

In [21]:
df_balanced = pd.concat([df_spam, df_ham_downsampled])
df_balanced.shape

(2998, 4)

In [22]:
df_balanced['label'].value_counts()

spam    1499
ham     1499
Name: label, dtype: int64

In [23]:
df_balanced['spam'] = df_balanced['label'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,Unnamed: 0,label,text,label_num,spam
1394,891,ham,Subject: new su bridge\r\nthe new sitara to un...,0,0
4634,1434,ham,Subject: feedback monitor error - meter 984132...,0,0
1297,4667,spam,"Subject: pleasure \ / all 1 um , ambl 1 en , x...",1,1
4672,4424,spam,"Subject: play on ocean treasure casino , get $...",1,1
1056,1750,ham,"Subject: re : meter 9707\r\nvance ,\r\ndid you...",0,0


In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_balanced['text'],df_balanced['spam'], stratify=df_balanced['spam'])

In [25]:
x_train.head()

1279                                        Subject: \r\n
2193    Subject: hpl noms for june 06 , 2000\r\n( see ...
982     Subject: re : fuel\r\nper mary , sitara can no...
4231    Subject: new product ! cialis soft tabs .\r\nh...
3065    Subject: re : atmic hurta # 1 - new production...
Name: text, dtype: object

## BERT

In [31]:
def get_sentence_embeding(sentence):
  preprocessed_text = bert_preprocess(sentence)
  return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
  "500$ discount. hurry up",
  "Jongmin, are you up for a tennis game tomorrow?"                       
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435169 , -0.5132725 , -0.88845736, ..., -0.7474887 ,
        -0.75314736,  0.91964495],
       [-0.88194007, -0.53932995, -0.9742502 , ..., -0.90923846,
        -0.76439774,  0.8939404 ]], dtype=float32)>

In [54]:
e = get_sentence_embeding([
  "I ate a banana this morning",
  "This is a good season for grape",
  "mango is tasty",
  "jeff bazos is a CEO of Amazon",
  "elon musk is a funny guy",
  "my name is jongmin baek and I live in Cary, NC"
])

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[0]], [e[5]])

array([[0.59679914]], dtype=float32)

In [64]:
# Bert_Layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network Layers
l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# construct final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])


In [66]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    